## Dynamic programming

Dynamic programming breaks a more complex problem down into smaller problems, making the overall problem simpler to solve. The solutions to these subproblems are saved and the larger problem is then easily solved when encountered. For this problem we assume we have an agent that want to learn the optimal policy for navigating the grid world where each action results in a reward of -1 and reaching a terminal state has a reward of 0. In other words, if the agent finds itself in any state, which action would it take to move towards the terminal state and maximize its rewards. The discount rate is set at 1, meaning future rewards are just as valuable as immediate rewards. To solve this problem we have the Bellman equation.

Bellman equation

Optimal policy $V^*$:
- $V^*(s) = max_{a} Q^*(s,a)$

Optimal state action pair (Q):
- $Q^*(s,a) = \sum_{s'} T(s,a,s') (R(s,a,s') + \gamma V^*(s'))$

Optimal policy $V^*$:
- $V^*(s) = max_a \sum_{s'} T(s,a,s') (R(s,a,s') + \gamma V^*(s'))$

Where:
- $T$ is the transition probability for moving from $s$, taking action $a$, to $s'$
    - Can also be expressed as $P(s'|s,a)$
- $R$ is the reward for being in state $s$, taking action $a$, and ending up in $s'$
- $\gamma$ is the discount factor
- $V^*(s')$ is the optimal policy from state $s'$, the state the agent moved to after taking action $a$


The discount factor affects the motivation of the agent to move towards the goal state, in this case the terminal state. Typically $0 < \gamma \leq 1$. If $\gamma = 1$ the agent sees future rewards just as valuable as immediate rewards. Depending on the reward structure this could lead to an agent not taking the optimal policy and meandering around the environment with no incentive to reach the goal. If $\gamma < 1$ future rewards are worth less the farther out they are, and again depending on the reward structure, the agent would seek to maximize rewards by moving to them quickly so as to not have a large discount factor reduce the rewards.

The equations are actually very simple. Here's an example of walking through $Q^*(s,a)$

This equation finds the value of the state, action pair. For example, the agent is in a state and wants to move north. The Q(s, north) tells us the value of moving north. All of these $Q(s,a)$ must be computed before we can find the $V^*(s)$. The value for terminal states is not updated.

Say we have a random policy, so for each square in a grid world we can move north, south, east, or west with an equal probability, so 25% chance. This becomes the transition probability. If the agent moves off the grid it just stays in place. To clarify, if the agent chooses the action to move north, there is a 25% chance of moving north, as well as a 25% chance of moving in one of the other three options (S,E,W).

For this example lets picture a very simple 3x3 grid with the agent starting in the center at (2,2). The rewards are 0 unless the agent is in a terminal state (3,3) where $R = 1$, and the discount factor is $\gamma = 1$.

In [1]:
import numpy as np
import copy

gw_ = np.zeros((3,3))
gw = np.zeros((3,3), dtype='<U5')
for i in range(3):
    for j in range(3):
        gw[i,j] = f"{i+1},{j+1}"
print("States\n",gw)
print("State Values\n",gw_)

States
 [['1,1' '1,2' '1,3']
 ['2,1' '2,2' '2,3']
 ['3,1' '3,2' '3,3']]
State Values
 [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]


$Q^*(s,a) = \sum_{s'} T(s,a,s') (R(s,a,s') + \gamma V^*(s'))$

North:
- $Q^*(s (2,2) , north) = 0.25 *  (-1 + 1* V^*(1,2))$
- $Q^*(s (2,2) , north) = 0.25 *  (-1 + 1* 0) = -0.25$

South:
- $Q^*(s (2,2) , south) = 0.25 *  (-1 + 1* V^*(3,2))$
- $Q^*(s (2,2) , south) = 0.25 *  (-1 + 1* 0) = -0.25$

East:
- $Q^*(s (2,2) , east) = 0.25 *  (-1 + 1* V^*(2,3))$
- $Q^*(s (2,2) , east) = 0.25 *  (-1 + 1* 0) = -0.25$

West:
- $Q^*(s (2,2) , west) = 0.25 *  (-1 + 1* V^*(2,1))$
- $Q^*(s (2,2) , west) = 0.25 *  (-1 + 1* 0) = -0.25$

Now we sum these values and return an overall value of $Q^*(s, north) = -1$

We need to do this again for $Q^*(s, south)$, $Q^*(s, east)$, and $Q^*(s, west)$. Because this is the first iteration and every state has been initialized to a value of 0 we get 0 for each state action pair. We then calculate $V^*(2,2)$ using the equation below.

$V^*(2,2) = max_a(Q(22, N),Q(22, S),Q(22, E),Q(22, W))$

$V^*(2,2) = max_a(-1, -1, -1, -1)$

$V^*(2,2) = -1$

In [2]:
gw_[1,1] = -1
print("States\n",gw)
print("State Values\n",gw_)

States
 [['1,1' '1,2' '1,3']
 ['2,1' '2,2' '2,3']
 ['3,1' '3,2' '3,3']]
State Values
 [[ 0.  0.  0.]
 [ 0. -1.  0.]
 [ 0.  0.  0.]]


Now say we moved south to state 3,2. We need to perform the above steps again, except that this time there is a reward of 1 in state 3,3, and we also have a value of -1 in state 2,2.

North:
- $Q^*(s (3,2) , north) = 0.25 *  (-1 + 1* V^*(2,2))$
- $Q^*(s (3,2) , north) = 0.25 *  (-1 + 1* -1) = -0.5$

South:
- The agent stays in place if it tries to move off the grid
- $Q^*(s (3,2) , south) = 0.25 *  (-1 + 1* V^*(3,2))$
- $Q^*(s (3,2) , south) = 0.25 *  (-1 + 1* 0) = -0.25$

East:
- State (3,3) is a terminal state and has a reward of 1
- The value of a  terminal state is not updated
- $Q^*(s (3,2) , east) = 0.25 *  (-1 + 1* V^*(3,3))$
- $Q^*(s (3,2) , east) = 0.25 *  (-1 + 1* 1) = 0$

West:
- $Q^*(s (3,2) , west) = 0.25 *  (-1 + 1* V^*(3,1))$
- $Q^*(s (3,2) , west) = 0.25 *  (-1 + 1* 0) = -0.25$

$Q^*((3,2), north) = -0.5+-0.25+0+-0.25 = -1$

We do this for each of the other actions (S,E,W) to get the following.

$Q^*((3,2), south) = -0.5+-0.25+0+-0.25 = -1$

$Q^*((3,2), east) = -0.5+-0.25+0+-0.25 = -1$

$Q^*((3,2), west) = -0.5+-0.25+0+-0.25 = -1$


Now we take the max of these values to update $V(3,2)$.

$V^*(3,2) = max_a(-1, -1, -1, -1) = -1$

This may not seem like much is happening, but what if we continue?

In [3]:
gw_[2,1] = -1
print("States\n",gw)
print("State Values\n",gw_)

States
 [['1,1' '1,2' '1,3']
 ['2,1' '2,2' '2,3']
 ['3,1' '3,2' '3,3']]
State Values
 [[ 0.  0.  0.]
 [ 0. -1.  0.]
 [ 0. -1.  0.]]


Now say we moved south to state 3,3. This is a terminal state where the reward is 1. We cannot select an action from a terminal state because the episode has ended. We could update the value of state (3,3) as follows, but typically it is just set to the reward value without bothering to calculate all of these equations as the result it simply the reward value.

North:
- $Q^*(s (3,2) , north) = 1.0 *  (1)$
- $Q^*(s (3,2) , south) = 1.0 *  (1)$
- $Q^*(s (3,2) , east) = 1.0 *  (1)$
- $Q^*(s (3,2) , west) = 1.0 *  (1)$

Now we take the max of these values to update $V(3,3)$.

$V^*(3,2) = max_a(1, 1, 1, 1) = 1$

When an agent finishes an episode by entering a terminal state the agent is typically reset. Here we'll reset to S(2,2) and recalculate the values we previously calculated. Below is the updated grid world with the new values.

In [4]:
gw_[2,2] = 1
print("States\n",gw)
print("State Values\n",gw_)

States
 [['1,1' '1,2' '1,3']
 ['2,1' '2,2' '2,3']
 ['3,1' '3,2' '3,3']]
State Values
 [[ 0.  0.  0.]
 [ 0. -1.  0.]
 [ 0. -1.  1.]]


Starting over in state (2,2)

North:
- $Q^*(s (2,2) , north) = 0.25 *  (-1 + 1* V^*(1,2))$
- $Q^*(s (2,2) , north) = 0.25 *  (-1 + 1* 0) = -0.25$

South:
- $Q^*(s (2,2) , south) = 0.25 *  (-1 + 1* V^*(3,2))$
- $Q^*(s (2,2) , south) = 0.25 *  (-1 + 1* -1) = -0.5$

East:
- $Q^*(s (2,2) , east) = 0.25 *  (-1 + 1* V^*(2,3))$
- $Q^*(s (2,2) , east) = 0.25 *  (-1 + 1* 0) = -0.25$

West:
- $Q^*(s (2,2) , west) = 0.25 *  (-1 + 1* V^*(2,1))$
- $Q^*(s (2,2) , west) = 0.25 *  (-1 + 1* 0) = -0.25$

Now we sum these values and return an overall value of $Q^*(s, north) = -1.25$

We need to do this again for $Q^*(s, south)$, $Q^*(s, east)$, and $Q^*(s, west)$. Because each action has a 25% of being taken for each possible direction we end up with the same values for each action again. We then calculate $V^*(2,2)$ using the equation below.

$V^*(2,2) = max_a(Q(22, N),Q(22, S),Q(22, E),Q(22, W))$

$V^*(2,2) = max_a(-1.25, -1.25, -1.25, -1.25)$

$V^*(2,2) = -1.25$

In [5]:
gw_[1,1] = -1.25
print("States\n",gw)
print("State Values\n",gw_)

States
 [['1,1' '1,2' '1,3']
 ['2,1' '2,2' '2,3']
 ['3,1' '3,2' '3,3']]
State Values
 [[ 0.    0.    0.  ]
 [ 0.   -1.25  0.  ]
 [ 0.   -1.    1.  ]]


Now we do state (3,2)

Now say we moved south to state 3,2. We need to perform the above steps again, except that this time there is a reward of 1 in state 3,3, and we also have a value of -1 in state 2,2.

North:
- $Q^*(s (3,2) , north) = 0.25 *  (-1 + 1* V^*(2,2))$
- $Q^*(s (3,2) , north) = 0.25 *  (-1 + 1* -1.25) = -0.3125$

South:
- The agent stays in place if it tries to move off the grid
- $Q^*(s (3,2) , south) = 0.25 *  (-1 + 1* V^*(3,2))$
- $Q^*(s (3,2) , south) = 0.25 *  (-1 + 1* 0) = -0.25$

East:
- State (3,3) is a terminal state and has a reward of 1
- The value of a  terminal state is not updated
- $Q^*(s (3,2) , east) = 0.25 *  (-1 + 1* V^*(3,3))$
- $Q^*(s (3,2) , east) = 0.25 *  (-1 + 1* 1) = 0$

West:
- $Q^*(s (3,2) , west) = 0.25 *  (-1 + 1* V^*(3,1))$
- $Q^*(s (3,2) , west) = 0.25 *  (-1 + 1* 0) = -0.25$

$Q^*((3,2), north) = -0.3125+-0.25+0+-0.25 = -0.8125$

Again, We do this for each of the other actions (S,E,W) and currently they have the same results, giving the following.

$Q^*((3,2), south) = -0.3125+-0.25+0+-0.25 = -0.8125$

$Q^*((3,2), east) = -0.3125+-0.25+0+-0.25 = -0.8125$

$Q^*((3,2), west) = -0.3125+-0.25+0+-0.25 = -0.8125$


Now we take the max of these values to update $V(3,2)$.

$V^*(3,2) = max_a(-0.8125,-0.8125,-0.8125,-0.8125) = -0.8125$

Below we can see the updated values. You can see that the values are beginning to converge in a way to moves the agent towards the terminal state. In the interest of not doing a ton of calculations we skipped the other states, but notice the ones we did the calculations for. If the agent want the maximum value as a reward, it could use the values of states to move from (2,2) to (2,3) and then to (3,3) (again, ignoring the states we didn't calculate yet). With enough iterations the values will converge to stable values that can be used in this way.

In [6]:
gw_[2,1] = -0.8125
print("States\n",gw)
print("State Values\n",gw_)

States
 [['1,1' '1,2' '1,3']
 ['2,1' '2,2' '2,3']
 ['3,1' '3,2' '3,3']]
State Values
 [[ 0.      0.      0.    ]
 [ 0.     -1.25    0.    ]
 [ 0.     -0.8125  1.    ]]


## Code

Mini grid world example for policy evaluation using dynamic programming from the book, Reinforcement Learning (Sutton chapter 4).

Solves policy evaluation and returns policy values and matrix with "arrows" showing which action to take next.

Gridworld is 4x4 with first and last cells being terminal states. The challenge is to find convergence of values for states through policy iteration. You can change the size of the grid world and whether there are 1 or 2 terminal states.

There are two possible terminal states, either the bottom right state, or the bottom right state and the top left state. Change this by using the kwargs = 'first_and_last' or omit to keep just the final state.

In [7]:
class GridWorldSolver():
    def __init__(self, rewards = -1, world_size = (4,4), gamma = 1, theta = 1e-5, **kwargs):
        self.r = rewards
        self.world = np.zeros((world_size[0], world_size[1]))
        self.g = gamma
        self.theta = theta
        self.terminals = kwargs.get('terminals', 'last')
        self.iterations = kwargs.get('iterations', 0)


    def v_pi(self, directions):
        """
        returns value for state
        params:
            directions:
        returns:
            total: sum of one_dir() which is return of all possible actions in that state
            (return from up, down, r, l x the prob of each (1/4) summed to represent the new v(s) for that space)
        """

        total = 0
        d = ['up','down','left','right']
        for i, direction in enumerate(directions):
            single_dir = (1/len(d)) * (self.r + self.g * self.world[direction])
            total += single_dir
        return total


    def calc_mini_gridworld(self):
        """calculates values of states until convergence
        params: None
        returns:
            self.world: array of updated values representing values of each state in grid
        """
        
        delta = np.inf
        
        if self.iterations:
            for i in range(self.iterations):
                self.iterate()
        else:
            i = 0
            while delta > self.theta:
                delta = self.iterate()
                i += 1
                if delta < self.theta:
                    break

        print(f"Iterations completed : {i+1}")

        return self.world
    

    def iterate(self):
        """iterates over states calculating and updating values with each iteration
        params: None
        returns:
            if not using fixed number of iterations:
                delta: change in values to compare against previous iteration to check for convergence
        """
        
        tmp_world = copy.deepcopy(self.world)

        for i in range(self.world.shape[0]):
            for j in range(self.world.shape[1]):

                if(i == self.world.shape[0]-1 and j == self.world.shape[1]-1):
                    pass
                elif (i == 0 and j == 0) and self.terminals == 'first_and_last':
                    pass

                else:
                    if i-1 < 0:
                        up = i
                    else:
                        up = i-1
                    if i+1 > self.world.shape[0] - 1:
                        down = i
                    else:
                        down = i+1
                    if j-1 < 0:
                        left = j
                    else:
                        left = j-1
                    if j+1 > self.world.shape[1] - 1:
                        right = j
                    else:
                        right = j+1

                    directions = [(up,j),(down,j), (i,left), (i,right)]
                    self.world[i,j] = self.v_pi(directions)

        if not self.iterations:
            w_flat = np.squeeze(self.world.reshape(1,-1))[1:]
            tmp_flat = np.squeeze(tmp_world.reshape(1,-1))[1:]
            delta = max(abs(w_flat - tmp_flat))
            return delta


    def add_arrows(self, grid_world):
        """translates numbers to actions"""

        action_world = np.zeros(grid_world.shape, dtype='<U2')
        for i in range(grid_world.shape[0]):
            for j in range(grid_world.shape[1]):
                if(i == self.world.shape[0]-1 and j == self.world.shape[1]-1):
                    action_world[i,j] = 'G'
                elif (i == 0 and j == 0) and self.terminals == 'first_and_last':
                    action_world[i,j] = 'G'
                else:
                    if i-1 < 0:
                        up = None
                    else:
                        up = i-1
                    if i+1 > grid_world.shape[0] - 1:
                        down = None
                    else:
                        down = i+1
                    if j-1 < 0:
                        left = None
                    else:
                        left = j-1
                    if j+1 > grid_world.shape[1] - 1:
                        right = None
                    else:
                        right = j+1
                    
                    directions = [(up,j),(down,j), (i,left), (i,right)]
                    d = ['up','down','left','right']
                    actions = ''
                    for k, direction in enumerate(directions):
                        try:
                            if grid_world[direction[0],direction[1]] > grid_world[i,j]:
                                if k == 0:
                                    actions += '^'
                                elif k == 1:
                                    actions += 'v'
                                elif k == 2:
                                    actions += '<' 
                                elif k == 3:
                                    actions += '>'
                                if len(actions) == 1:
                                    actions += ' '
                        except:
                            pass

                    action_world[i,j] = actions
        return action_world


In [8]:
def main(**kwargs):
    terminals = kwargs.get('terminals','last')
    iterations = kwargs.get('iterations', 0)
    mini_grid_world = GridWorldSolver(terminals = terminals, 
                                      iterations = iterations)
    grid_world = mini_grid_world.calc_mini_gridworld()
    print(grid_world.round(1))
    diagram = mini_grid_world.add_arrows(grid_world)
    print(diagram)

In [9]:
main()

Iterations completed : 383
[[-59.4 -57.4 -54.3 -51.7]
 [-57.4 -54.6 -49.7 -45.1]
 [-54.3 -49.7 -40.9 -30. ]
 [-51.7 -45.1 -30.    0. ]]
[['v ' 'v ' 'v ' 'v ']
 ['v ' 'v ' 'v ' 'v ']
 ['v ' 'v ' 'v ' 'v ']
 ['> ' '> ' '> ' 'G']]


In [10]:
main(terminals = 'first_and_last', iterations = 100)

Iterations completed : 100
[[  0. -14. -20. -22.]
 [-14. -18. -20. -20.]
 [-20. -20. -18. -14.]
 [-22. -20. -14.   0.]]
[['G' '< ' '< ' 'v ']
 ['^ ' '^ ' '^ ' 'v ']
 ['^ ' '^ ' 'v ' 'v ']
 ['^ ' '^ ' '> ' 'G']]
